
 
 
 ## ASML Induction: Introduction to Jupyter and R

 
In the Advanced Statistics and Machine Learning modules, we will use R for programming and data analysis. 

This document is a Jupyter R notebook. It can be used for simultaneous coding (in R) and typesetting (in Markdown).

This is not the only way of using R.  Another option is to use R Studio, which can be run either locally (based on a local installation on your laptop), or online (via R Studio Cloud), or remotely (on the Durham CIS System via AppsAnywhere), or, actually, also in some way via Jupyter.

The plain Jupyter version of using R - as illustrated here and today - will be used in ASML for the Foundations and Unsupervised Learning submodules. 

R Studio will be used in the ASML Regression and Classification submodules.


**Objectives of this induction**

* using Jupyter Notebooks;

* reading in data;

* working with vectors, matrices, and data frames; 

* basic programming devices (such as if...then, for, while, apply, functions); see also the  [Handout](http://www.maths.dur.ac.uk/~dma0je/PG/Mix/MSc/ASMLHandout.pdf).

* application to real data sets.

* basic clustering.


Solutions will be provided following the induction, which can be picked up from Jupyter the same way as you have fetched this notebook.



**Preliminaries**

As mentioned above there are two types of content in a Jupyter notebook: Code and Markdown.  This here is a Markdown cell. The following cell is a code cell. Execute the code in that cell, by clicking either on the black triangle symbol or on "Run", and try to understand what the code does.


In [ ]:
x <-3
x
D <- date()
D
DayofWeek<- substr(D, 1,x)     # extracts the first x letters from date object
cat("Today's day of the week is:", DayofWeek)



When you define a new cell, you need to decide on whether it is a Code cell or a Markdown cell.  You can create new cells by firstly clicking "Insert" from the menu bar (and then Insert cell below / Insert cell above), and then defining the type of content in the curresponding drop-down menu (with options Markdown and Code, among some others). Do this now. Define a new Markdown cell with content "This is my first Markdown cell." and then a Code cell which computes and displays the square of the variable `x`.

This is my first Markdown cell.

In [ ]:
x^2

**Reading in Data**

The first data set that we are going to investigate gives the energy use (kg of oil equivalent per capita) over 135 countries from 1960 to 2010.

*Energy use* is defined as the use of primary energy before transformation to other end-use fuels, which is equal to indigenous production plus imports and stock changes, minus exports and fuels supplied to ships and aircraft engaged in international transport.

Source: [Worldbank](http://data.worldbank.org/indicator/EG.USE.PCAP.KG.OE)

You can read the data in via 

In [ ]:
energy.use <- read.csv("http://www.maths.dur.ac.uk/~dma0je/Data/energy.csv", header=TRUE)

If things have gone right, then

In [ ]:
dim(energy.use)

should  give you the dimension $135 \times 12$.  Also, visualize the first six rows via

In [ ]:
head(energy.use)

**Working with vectors, matrices and data frames**

The object `energy.use` is a *data frame*.  You can check whether or not an object is a data frame by typing `class(object)` or `is.data.frame(object)`. Try this for the object `energy.use` in the chunk below.


In [ ]:
class(energy.use)          
is.data.frame(energy.use)   

It is easy to access individual rows, columns, or elements of a data frame. For instance,

In [ ]:
 energy.use[127,]
 energy.use[,49]
 energy.use[127,49] 

will give you the 127th row; 49th column; and the entry of the 127th row and the 49th column, respectively (this is the UK energy consumption in 2007). You can also access columns directly through their column names, such as

In [ ]:
energy.use$X2007

Data frames are very important as they are the standard form in which data are expected by many R functions, such as
`lm`, `glm`,....

Let us now simplify the data frame a little bit, so that it is easier to use for the applied work. We reduce our interest to the energy consumption
in the years 2001 and 2007. We do this via

In [ ]:
 energy <- energy.use[,c("X2001", "X2007")]

Also, we would like to give the rows and columns of the new data frame meaningful names.  Please type 

In [ ]:
rownames(energy)<- energy.use[, 1]
colnames(energy)<- c("use01", "use07")


in order to specify row and column names, respectively.  Then type `energy` to look at your final data frame.  

This data frame allows to access information quickly. For instance,

In [ ]:
 energy["United Kingdom",]

gives you the UK values of energy consumption. **DO THIS** for a couple of countries.

In [ ]:
 energy["Spain",] 
 energy["China",]


**Basic programming devices**

One may be interested in looking at these data in a form in which they are *ordered* by their energy consumption.  This can be done using


In [ ]:
 order(energy$use07)

which gives you a list of numbers. The first number tells you the index (here: 39) of the country with the smallest per-capita energy consumption (here: Eritrea), and typing `energy[order(energy$use07),]`
gives you the full ordered list. 

In the chunk below, save this ordered data frame into a new data frame `senergy`.

In [ ]:
senergy <- energy[order(energy$use07),] 

Next, we wish to identify the nations with extremely large  energy consumption, say, more than 10000 kg of oil per capita  (Intuitively, what do you think, which countries will this be?). Calling

In [ ]:
energy$use07 > 10000

will give you a vector of logical values, with a  `TRUE`  for each country for which this condition is met. The command

In [ ]:
sum(energy$use07 > 10000)

will tell you how many these are, and


In [ ]:
 which(energy$use07 > 10000)

will give you the index numbers of these countries. From this, we would get the data rows corresponding to these countries via

In [ ]:
 energy[which(energy$use07 > 10000),]

We would like to compare the energy use in 2001 and 2007. Do the same as above but now use the condition `energy$use01 > energy$use07` instead.  Observe and understand the information that you gain at each step.

In [ ]:
energy$use01> energy$use07            
sum(energy$use01> energy$use07)       
which(energy$use01> energy$use07)     
energy[which(energy$use01> energy$use07),] 

A very useful tool to carry out repeated operations is the `for` command (see [Handout](http://www.maths.dur.ac.uk/~dma0je/PG/Mix/MSc/ASMLHandout.pdf)!).

Task: Implement a loop which, for all 135 countries, writes a text  like

<center>
 In 2007,  the energy use in *country* was equivalent to *value*   kg oil per capita.
</center>&nbsp;

In [ ]:
 for (i in 1:135){   
   cat("In 2007, the energy use in ", rownames(energy)[i], " was equivalent to",  energy[i,2], "kg oil per capita.", "\n")
  } 


Another command for repeated operations is `while`. It does not have a fixed number of loops, but proceeds until a certain condition is met. For instance, consider the ordered frame `senergy` created above. Assume we are interested in the following question: If we take exactly one person from each of the countries with the smallest energy use, i.e. one person from  Eritrea, one person from Bangladesh, etc., then how many persons are needed in order to achieve the same use of energy as a single person in Qatar?

To answer this, create objects `i` and `sum07` and assign them the initial value 0. Then use the `while` function (see [Handout](http://www.maths.dur.ac.uk/~dma0je/PG/Mix/MSc/ASMLHandout.pdf)) with *condition* `sum07< senergy["Qatar",2]` and *action* `i     <- i+1; sum07 <- sum07+  senergy[i,2]`.  Make it clear to yourself what each row does. Also, interpret the result.


In [ ]:
 energy["Qatar",]      
  i     <-0             
  sum07 <-0             
  while(sum07< senergy["Qatar",2] ){
     i=i+1                          
     sum07<- sum07+  senergy[i,2]   
  }                                 
  i                                 
  sum07                             
  # So individuals from the 41 least-consuming countries use less energy per captita than one single individual in Qatar!  


Use `apply` to compute a vector which contains, for each country, the larger of the two energy consumption values given for 2001 and 2007. Consult the see [Handout](http://www.maths.dur.ac.uk/~dma0je/PG/Mix/MSc/ASMLHandout.pdf) and the corresponding help file (via `help(apply)` or `?apply`) if you are unsure how to do this.


In [ ]:
apply(energy,1,max)

**Basic data visualization**

Use `hist` and `boxplot` to create  histograms and boxplots of the variables `use01` and `use07`. Comment on the distributional shape.

In [ ]:
 boxplot(energy$use01, energy$use07)   
 par(mfrow=c(2,1))                     
 hist(energy$use01)                    
 hist(energy$use07) 

Next, add logarithmic versions of these variables, say `luse01` and`luse07`,  to the data frame via


In [ ]:
energy$luse01<- log(energy$use01)

and for`use07`  analogously. Repeat the previous question using the transformed variables. What can we say about the distribution of these transformed variables, compared to the original ones?

In [ ]:
  energy$luse07 <- log(energy$use07)   
  boxplot(energy$luse01, energy$luse07)
  par(mfrow=c(2,1))                    
  hist(energy$luse01)                  
  hist(energy$luse07)   

**Simple clustering via k-means**

Next, we consider a data set featuring $n=82$ observations of galaxy velocities. Load the `galaxies` data, read the associated help file, and create a histogram using the option `breaks =18` in function `hist`.

In [ ]:
 data(galaxies, package="MASS")  
 ?galaxies
hist(galaxies, breaks=18)

For both data sets, the dominating feature is the presence of multiple modes or `clusters'. It is a relevant problem in Statistics and Machine Learning to identify such clusters, and also find the corresponding cluster centers. A simple method to do this is the **k-means algorithm**. See for instance [this resource](https://bigdata-madesimple.com/possibly-the-simplest-way-to-explain-k-means-algorithm/) for a quick introduction into this algorithm.

In R, this algorithm is implemented in  the function `kmeans`. The algorithm requires the specification of the number of clusters in advance, through the argument `centers`.  Study the help file of `kmeans` and then apply this function  onto the `luse01` and `galaxies` data. You will need only the first two arguments of `kmeans`. For the choice of the number of clusters, you can use visual inspection as a guide for your choice. 

In [ ]:
?kmeans
kmeans(energy$luse01, centers=2)
kmeans(galaxies, centers=5)

Have a look at the produced output and try to understand and interpret it in the light of the graphical representations of the data presented earlier.

This was an example for the application a very simple clustering technique, in the one-dimensional (univariate) case.

In the "Unsupervised Learning" submodule, we will pick up from here, and consider more advanced clustering techniques, as well as clustering in higher dimensions.

The "Regression" and "Classifications" submodules will be set in the world of *supervised learning*, where training samples with true and known class labels (or output values) are available.


**Further resources**

If you would like to do do a bit more to improve your R skills, we recommend the following resources:

* [https://education.rstudio.com/](https://education.rstudio.com/)
* [https://www.jaredknowles.com/r-bootcamp](https://www.jaredknowles.com/r-bootcamp)



**Thanks for your participation, and see you in January!**